# Multimodal Search Cookbook with Seed1.8


This cookbook implements a **Multimodal Search Agent** using the **Seed1.8 model**. It integrates web search, link reading, and image/visual search tools to perform multimodal retrieval and summarization.

### Core Features

1. Single-Turn Inference
- Processes a single user query and provides a response.
- Supports tool integration for online search and content summarization.

2. Multi-Turn Inference
- Handles multi-step reasoning to accomplish complex tasks.
- Manages conversation history using a memory condenser to optimize performance.

3. Memory Management
- Implements a FIFO-based memory condenser to retain the most appropriate conversation history during multi-turn inference.

4. Force Answer
- When max steps is reached, the agent will be forced to provide an answer.
---


### 0. Prerequisites
- **API Key**: [Obtain an API key](https://console.volcengine.com/ark/region:ark+cn-beijing/apiKey) for the Seed1.8 model.
- **Model Endpoint**: Ensure the base URL of the Ark API.
- **Activate the Model**: Activate the Seed1.8 model in [the Ark Console](https://console.volcengine.com/ark/region:ark+cn-beijing/openManagement).
- **Install Dependencies**: Install required Python packages.


In [ ]:
pip install -r ../requirements.txt

In [3]:
# Copyright 2025 Bytedance Ltd. and/or its affiliates.
# SPDX-License-Identifier: Apache-2.0

# Please set the API key here
import os

os.environ['ARK_API_KEY']  = 'your_ark_api_key'
os.environ['ARK_MODEL_ENDPOINT'] = "doubao-seed-1-8-251215"
base_url = "https://ark.cn-beijing.volces.com/api/v3"



### 1. Define Tool Schemas
Definitions of tools such as Search, ScholarSearch, LinkReader, Fetch, and ImageSearch. 


In [5]:
# tool schemas
tool_schemas = [
  {
    "type": "function",
    "function": {
      "name": "Search",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "搜索问题"
          }
        },
        "required": [
          "query"
        ]
      },
      "description": "这是一个联网搜索工具，输入搜索问题，返回网页列表与对应的摘要信息。搜索问题应该简洁清晰，复杂问题应该拆解成多步并一步一步搜索。如果没有搜索到有用的页面，可以调整问题描述（如减少限定词、更换搜索思路）后再次搜索。搜索结果质量和语种有关，对于中文资源可以尝试输入中文问题，非中资源可以尝试使用英文或对应语种。"
    }
  },
  {
    "type": "function",
    "function": {
      "name": "ScholarSearch",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "学术搜索问题"
          }
        },
        "required": [
          "query"
        ]
      },
      "description": "这是一个学术文献搜索工具，输入学术搜索问题，返回相关的学术论文、研究报告等文献信息。搜索问题应该简洁清晰，复杂问题应该拆解成多步并一步一步搜索。适用于查找专业研究成果，论文等。如果没有搜索到有用的页面，可以调整问题描述（如减少限定词、更换搜索思路）后再次搜索。搜索结果质量和语种有关，对于中文资源可以尝试输入中文问题，非中资源可以尝试使用英文或对应语种。"
    }
  },
  {
    "type": "function",
    "function": {
      "name": "LinkReader",
      "description": "这是一个链接浏览工具，可以打开链接（可以是网页、pdf等）并根据需求描述汇总页面上的所有相关信息。建议对所有有价值的链接都调用该工具来获取信息，有价值的链接包括但不限于如下几种：1.任务中明确提供的网址，2.搜索结果提供的带有相关摘要的网址（图片搜索返回的图片来源网址也在此范围中），3. 之前调用LinkReader返回的内容中包含的且判断可能含有有用信息的网址。请尽量避免自己凭空构造链接。",
      "parameters": {
        "properties": {
          "url": {
            "type": "string",
            "description": "目标链接，应该是一个完整的url（以 http 开头）"
          },
          "description": {
            "type": "string",
            "description": "需求描述文本，详细描述在当前url内想要获取的内容"
          }
        },
        "required": [
          "url",
          "description"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "Fetch",
      "description": "这是一个强大的页面与文件内容获取工具。根据输入的链接获取对应的详细内容，包括中的页面与文件内容中的图片信息。支持解析在线网页、PDF等多种文本文件格式。工具提供两种访问模式：1.pagination 模式：可以通过分页能力 pagination 配合offset和limit实现分片段读取全文内容。2.snippet模式：必须设置query，会基于query精简提取文件中与需求相关的内容snippet，如果query为空，直接返回读取的全文content。",
      "parameters": {
        "type": "object",
        "properties": {
          "url": {
            "type": "string",
            "description": "要获取的文档URL"
          },
          "reference_id": {
            "type": "string",
            "description": "引用ID（可选）"
          },
          "use_pagination": {
            "type": "boolean",
            "description": "是否使用分页模式",
            "default": False
          },
          "offset": {
            "type": "integer",
            "description": "分页偏移量（当use_pagination=true时生效）",
            "default": 0
          },
          "limit": {
            "type": "integer",
            "description": "读取长度限制",
            "default": 4000
          },
          "use_snippet": {
            "type": "boolean",
            "description": "是否使用snippet模式",
            "default": False
          },
          "query": {
            "type": "string",
            "description": "搜索查询（当use_snippet=true时必填）"
          },
          "snippet_max_length": {
            "type": "integer",
            "description": "snippet最大长度",
            "default": 4000
          },
          "snippet_mode": {
            "type": "string",
            "description": "snippet模式：full/minimum",
            "enum": [
              "full",
              "minimum"
            ],
            "default": "full"
          },
          "thumbnail_size": {
            "type": "string",
            "description": "图片返回大小：SMALL/MEDIUM/LARGE",
            "enum": [
              "SMALL",
              "MEDIUM",
              "LARGE"
            ],
            "default": "SMALL"
          }
        },
        "required": [
          "url"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "ImageSearch",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "搜索问题"
          }
        },
        "required": [
          "query"
        ]
      },
      "description": "这是一个联网搜索工具，输入搜索问题，返回图片链接列表与对应的原始网页url等信息。搜索问题应该简洁清晰，复杂问题应该拆解成多步并一步一步搜索。如果没有搜索到有用的页面，可以调整问题描述（如减少限定词、更换搜索思路）后再次搜索。搜索结果质量和语种有关，对于中文资源可以尝试输入中文问题，非中资源可以尝试使用英文或对应语种。"
    }
  },
  {
    "type": "function",
    "function": {
      "name": "visual_search",
      "description": "这是一个视觉搜索工具，用于从全网搜索到与输入图片相似的图片结果。基于图片进行搜索，返回相关图片、视频和产品信息。",
      "parameters": {
        "type": "object",
        "properties": {
          "reference_id": {
            "type": "string",
            "description": "作为搜索输入的图片标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识，例如看到'<image: 0>'就输入'<image: 0>'，看到'<image: 2>'就输入'<image: 2>'。"
          },
          "region_of_interest": {
            "type": "string",
            "description": "重点关注的图片区域，**必须**是字符串格式的数组，包含4个浮点数值（0-1之间），表示关注区域在全图中的归一化坐标。格式：'[min_x, min_y, max_x, max_y]'。示例：'[0.15, 0.0, 0.85, 0.55]'表示从左15%到右85%、从顶部0%到55%的区域。不提供则默认为全图'[0, 0, 1, 1]'。注意：必须用字符串格式，不能直接传数组。"
          },
          "thumbnail_size": {
            "type": "string",
            "description": "返回图片大小，可选：tiny/small/medium/large；分别对应图片的图片长边像素为：112/224/448/896。",
            "enum": [
              "tiny",
              "small",
              "medium",
              "large"
            ],
            "default": "medium"
          }
        },
        "required": [
          "reference_id"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "ZoomIn",
      "description": "图片放大工具。裁剪/放大指定区域，以便查看图片细节或文字识别或继续后续识别，对于图片中找东西、搜索图片中的东西非常有用。ZoomIn后的思考部分需要对ZoomIn出来的图片做详细解释。",
      "parameters": {
        "type": "object",
        "properties": {
          "imgidx": {
            "type": "string",
            "description": "待放大图片的标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识，例如看到'<image: 0>'就输入'<image: 0>'，看到'<image: 2>'就输入'<image: 2>'。"
          },
          "coord": {
            "type": "string",
            "description": "ROI坐标，**必须**是字符串格式的数组，包含4个浮点数值（0-1之间），表示裁剪区域的归一化坐标。格式：'[left_top_x, left_top_y, right_bottom_x, right_bottom_y]'。示例：'[0.1, 0.1, 0.9, 0.9]'表示从左上角10%到右下角90%的区域。注意：必须用字符串格式，不能直接传数组。禁止使用'[0, 0, 1, 1]'这种无意义的全图坐标。"
          }
        },
        "required": [
          "imgidx",
          "coord"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "Rotation",
      "description": "图片旋转工具。可以按指定角度旋转图片，支持任意角度旋转。常用于纠正图片方向或创建旋转效果。",
      "parameters": {
        "type": "object",
        "properties": {
          "imgidx": {
            "type": "string",
            "description": "待旋转图片的标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识。"
          },
          "angle": {
            "type": "number",
            "description": "旋转角度（度），正数为逆时针旋转，负数为顺时针旋转。常用值：90, 180, 270, -90等。"
          },
          "expand": {
            "type": "boolean",
            "description": "是否扩展画布以容纳旋转后的完整图片，默认为True。False时会裁剪到原始尺寸。",
            "default": True
          }
        },
        "required": [
          "imgidx",
          "angle"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "BrightnessContrast",
      "description": "图片亮度和对比度调整工具。可以增强或减弱图片的亮度和对比度，用于改善图片的视觉效果或便于识别。",
      "parameters": {
        "type": "object",
        "properties": {
          "imgidx": {
            "type": "string",
            "description": "待调整图片的标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识。"
          },
          "brightness": {
            "type": "number",
            "description": "亮度调整因子。1.0为原始亮度，大于1.0增加亮度，小于1.0降低亮度。建议范围：0.5-2.0。",
            "default": 1.0
          },
          "contrast": {
            "type": "number",
            "description": "对比度调整因子。1.0为原始对比度，大于1.0增加对比度，小于1.0降低对比度。建议范围：0.5-2.0。",
            "default": 1.0
          }
        },
        "required": [
          "imgidx"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "Flip",
      "description": "图片翻转工具。可以水平或垂直翻转图片，用于纠正镜像或创建翻转效果。",
      "parameters": {
        "type": "object",
        "properties": {
          "imgidx": {
            "type": "string",
            "description": "待翻转图片的标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识。"
          },
          "direction": {
            "type": "string",
            "description": "翻转方向。'horizontal'或'h'表示水平翻转（左右翻转），'vertical'或'v'表示垂直翻转（上下翻转）。",
            "enum": [
              "horizontal",
              "vertical",
              "h",
              "v"
            ]
          }
        },
        "required": [
          "imgidx",
          "direction"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "PutBox",
      "description": "在图片上添加标记框工具。可以在图片的指定区域绘制矩形框，用于标记、高亮或注释图片中的特定区域。",
      "parameters": {
        "type": "object",
        "properties": {
          "imgidx": {
            "type": "string",
            "description": "待添加标记框的图片标识。**必须**使用完整格式如'<image: 0>'、'<image: 2>'等，不能使用简单数字'0'、'2'。请直接复制对话历史中看到的图片标识。"
          },
          "coord": {
            "type": "string",
            "description": "标记框坐标，**必须**是字符串格式的数组，包含4个浮点数值（0-1之间），表示框区域的归一化坐标。格式：'[left_top_x, left_top_y, right_bottom_x, right_bottom_y]'。示例：'[0.1, 0.1, 0.9, 0.9]'表示从左上角10%到右下角90%的区域。注意：必须用字符串格式，不能直接传数组。"
          },
          "box_color": {
            "type": "string",
            "description": "标记框颜色。支持颜色名称（如'red', 'blue', 'green'）或十六进制颜色码（如'#FF0000'）。",
            "default": "red"
          },
          "box_width": {
            "type": "integer",
            "description": "标记框线条宽度，像素单位。建议范围：1-10。",
            "default": 3
          }
        },
        "required": [
          "imgidx",
          "coord"
        ]
      }
    }
  }
]

### 2. Prepare Inputs
User prompts and (optionally) images.


In [ ]:

import json
from PIL import Image
import io
import base64
import requests

def single_turn_inference(messages, tool_schemas=[]):

    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    data = {
        "model": model,
        "messages": messages,
        "max_tokens": 4096,
        "top_p": 0.9,
        "tools": tool_schemas,
        "temperature": 0.0,
        "reasoning_effort": "medium"
    }
    
    response = requests.post(base_url, headers=headers, json=data)
    try:
        print(f"Response: {response.json()}")
    except:
        print("Error fetching response.json:")
        print(response)
        
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.json())
        return {
            "error": f"Request failed with status code {response.status_code}",
            "details": response.text
        }

def pil_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str   

base_url = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
api_key = os.getenv('ARK_API_KEY')
model = os.getenv('ARK_MODEL_ENDPOINT')

image = Image.open("test.png")
base64_image = pil_to_base64(image)
messages = [
    {
        "role": "system",
        "content": "You are an agent designed to accomplish tasks.\n\nYour goal is to provide a comprehensive, objective, and well-researched answer to the user's query. You have access to multiple tools. Please read each tool's description and parameters carefully. There is no limit on how many times tools can be used. Please reason and respond in the same language as the user's question, unless the user explicitly requests otherwise. Put your final answer between <answer> and </answer>.\n"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": 'The following image is: <image: 0>\nType: User uploaded image',
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"
                }
            }
        ]
    },
    {
        "role": "user",
        "content": 'Please answer the following question and also provide your problem-solving roadmap. Question: The image is a photo of a stadium. In a video titled containing "Premium Experience" published on the stadium\'s official YouTube channel in February 2016, there\'s a scene where fans are cheering for a hurling goal. What is the jersey number of the player who scored the goal?'
    },
]

### 4. Single-Turn Inference


In [7]:
# Single-turn Model inference
response = single_turn_inference(messages, tool_schemas)

# Output model prediction results
print("\n" + "=" * 40)
print("🌟 🌟 🌟  **Model Prediction**  🌟 🌟 🌟")
print("=" * 40)
print(json.dumps(response, indent=4))  # Format JSON output

# Parse tool calls
actions = []
print("\n" + "=" * 40)
print("🔧 **Tool Calls** 🔧")
print("=" * 40)
for tool_call in response["choices"][0]["message"]["tool_calls"]:
    print("\n📌 Tool Call:")
    print(json.dumps(tool_call, indent=4, ensure_ascii=False))  # Format JSON output
    function_name = tool_call["function"]["name"]
    function_arguments = tool_call["function"]["arguments"]
    function_arguments = json.loads(function_arguments)
    actions.append(
        {
            "action_type": function_name,
            "action_inputs": function_arguments
        }
    )

Response: {'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'reasoning_content': "Got it, let's tackle this step by step. First, I need to figure out which stadium this is. Let's start by identifying the stadium from the image. The image has SuperValu ads, section 21, looks like a GAA stadium maybe? Let's do a visual search first.", 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"reference_id":"<image: 0>","region_of_interest":"[0.0, 0.0, 1.0, 1.0]"}', 'name': 'visual_search'}, 'id': 'call_bhzyzy2ttl4yqptbgr98lda0', 'type': 'function'}]}}], 'created': 1765462398, 'id': '0217654623945073393bc98fc45d4a5de2d8a04f79b33efa5a52b', 'model': 'doubao-seed-1-8-preview-251115', 'service_tier': 'default', 'object': 'chat.completion', 'usage': {'completion_tokens': 123, 'prompt_tokens': 7371, 'total_tokens': 7494, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 64}}}

🌟 🌟 🌟  **Mode

### 5. Multi-Turn Inference

Multi-turn inference handles complex tasks by running an iterative loop where the agent plans, calls tools, and integrates results across multiple steps. This approach is ideal for scenarios requiring decomposition, such as chaining web searches, reading multiple links for evidence gathering, or performing staged visual operations like zoom-in and crop.

Unlike single-turn inference, which makes only one model call, the multi-turn process repeatedly invokes the model with an expanding context that includes prior tool outputs. This enables deeper reasoning and yields more complete, robust answers, though at the cost of higher latency and token usage. The loop is managed with controls like `max_steps` to limit iterations and a FIFO-based memory condenser to keep the conversation history concise.
Use this when the task requires decomposition, chained browsing/reading, staged visual operations (e.g., zoom-in, box/region focus), or progressive summarization.


In [ ]:
# Multi-turn Model inference

class FIFOCondenser:
    def __init__(self, max_queue_length: int = 9999, min_queue_length: int = 1) -> None:
        super().__init__()
        self.max_queue_length = max_queue_length
        self.min_queue_length = min_queue_length

    def condense(
            self, conversations: list[dict], **kwargs
    ) -> list[dict]:
        if len(conversations) <= self.max_queue_length:
            return conversations

        new_conversations = []
        num_2b_popped = self.max_queue_length - self.min_queue_length
        for conversation_id, conversation in enumerate(conversations):
            if conversation["role"] in ["system", "user"]:
                new_conversations.append(conversation)
                continue
            if num_2b_popped > 0:
                num_2b_popped -= 1
                continue
            new_conversations.append(conversation)

        return new_conversations

def multi_turn_inference(instruction, image_path, max_steps=5):
    """
    Perform multi-turn inference to accomplish a task based on the given instruction.

    Args:
        instruction (str): The user's query or task instruction.
        max_steps (int): Maximum number of steps allowed for the inference process.

    Returns:
        str: The final answer provided by the agent.
    """
    # Initialize the conversation with system and user messages
    all_images = []
    image = Image.open(image_path)
    base64_image = pil_to_base64(image)
    all_images.append(base64_image)
    messages = [
        {
            "role": "system",
            "content": "You are an agent designed to accomplish tasks.\n\nYour goal is to provide a comprehensive, objective, and well-researched answer to the user's query. You have access to multiple tools. Please read each tool's description and parameters carefully. There is no limit on how many times tools can be used. Please reason and respond in the same language as the user's question, unless the user explicitly requests otherwise. Put your final answer between <answer> and </answer>.\n"
        },
        {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": 'The following image is: <image: 0>\nType: User uploaded image',
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"
                }
            }
            ]
        },
        {
            "role": "user",
            "content": instruction
        },
    ]

    # Initialize variables
    finished = False
    cur_step = 0
    answer = None
    fifo_condenser = FIFOCondenser(max_queue_length=20, min_queue_length=5)  # Short-term memory condenser, the best practice is max_queue_length=30, min_queue_length=20

    while not finished and cur_step < max_steps:
        print(f"\n{'=' * 40}")
        print(f"🔄 Step {cur_step + 1}/{max_steps}")
        print(f"{'=' * 40}")

        # If it's the last step, force the model to output the final answer
        if cur_step + 1 == max_steps:
            messages.append(
                {
                    "role": "user",
                    "content": (
                        "You must absolutely not perform any FunctionCall, tool invocation, Search, Open_url, code execution, or similar actions. "
                        "You can only answer the original question based on the information already retrieved and your own internal knowledge. "
                        "Please put your final answer only inside <answer>...</answer> tags, and do not output anything else outside the tags. "
                        "If you attempt to call any tool or FunctionCall, it will be considered a mistake."
                    )
                }
            )

        # Condense short-term memory to manage message history
        condensed_messages = fifo_condenser.condense(messages)

        # Perform single-turn inference
        response = single_turn_inference(condensed_messages, tool_schemas)
        messages.append(response["choices"][0]["message"])  # Append the model's response to the conversation history

        # Check the finish reason to determine the next action
        finish_reason = response["choices"][0]["finish_reason"]
        if finish_reason == "stop":
            # The model has provided the final answer
            finished = True
            answer = response["choices"][0]["message"]["content"].replace("</answer>", "")
            print("✅ Final answer received.")
        elif finish_reason == "tool_calls":
            # The model has requested tool usage
            tool_calls = response["choices"][0]["message"]["tool_calls"]
            tool_response = ""
            for tool_call in tool_calls:
                tool_name = tool_call["function"]["name"]
                tool_args = json.loads(tool_call["function"]["arguments"])
                tool_call_id = tool_call["id"]
                # Mock tool response (replace with actual tool execution if needed)
                tool_response += f"🔧 Tool response from: {tool_name}({json.dumps(tool_args, ensure_ascii=False)})\n"
            messages.append({"role": "tool", 
            "content": [
                {
                    "type": "text",
                    "text": tool_response,
                }, 
                # some tool may return image, mock the image url
                {
                    "type": "text",
                    "text": 'The following image is: <image: {image_id}>'.format(image_id=len(all_images))
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"
                }
                }, 
                ],
            "tool_call_id": tool_call_id})
            print(f"🔧 Tool call made: {tool_response}")
            all_images.append(base64_image)
        else:
            # Handle unknown finish reasons
            print(f"⚠️ Unknown finish_reason: {finish_reason}")
            break

        # Print the response for debugging
        print(f"📝 Response: {json.dumps(response, indent=4, ensure_ascii=False)}")
        cur_step += 1

    # Final output
    if finished:
        assert answer is not None
        print(f"\n🎉 Task completed in {cur_step} steps!")
        print(f"✅ Final Answer: {answer}")
    else:
        print(f"\n❌ Failed to complete the task within {cur_step} steps.")

    return answer


# Example usage
instruction = 'Please answer the following question and also provide your problem-solving roadmap. Question: The image is a photo of a stadium. In a video titled containing "Premium Experience" published on the stadium\'s official YouTube channel in February 2016, there\'s a scene where fans are cheering for a hurling goal. What is the jersey number of the player who scored the goal?'
image_path = "test.png"

answer = multi_turn_inference(instruction, image_path)


🔄 Step 1/5
Response: {'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'reasoning_content': "Got it, let's tackle this step by step. First, I need to figure out which stadium this is. Let's start by identifying the stadium from the image. The image has SuperValu ads, section 21, looks like a GAA stadium maybe? Let's do a visual search first.", 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"reference_id":"<image: 0>","region_of_interest":"[0.0, 0.0, 1.0, 1.0]"}', 'name': 'visual_search'}, 'id': 'call_gtjefacyyr9a7tb8p2tcmhru', 'type': 'function'}]}}], 'created': 1765462415, 'id': '0217654624077004b4b7d656bffcdc9cfc98fb1f23cda5a9180e6', 'model': 'doubao-seed-1-8-preview-251115', 'service_tier': 'default', 'object': 'chat.completion', 'usage': {'completion_tokens': 123, 'prompt_tokens': 7371, 'total_tokens': 7494, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 64}}}
🔧 

## Best Practices

